In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os
import pickle

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append("/home/muncok/DL/projects/")

In [3]:
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F
from sv_system.data.dataloader import init_default_loader
from sv_system.utils.parser import get_sv_parser
from sv_system.train.si_train import set_seed
from sv_system.data.dataset import SpeechDataset

ModuleNotFoundError: No module named 'sv_system'

In [ ]:
from sv_system.utils import secToFrames, secToSample
from sv_system.utils.parser import test_config
si_config = test_config('tdnn')
si_config['input_clip'] = True
si_config['input_length'] = secToSample(3)
si_config['input_frames'] = secToFrames(3)
si_config['splice_frames'] = secToFrames(0.1)
si_config['input_format'] = 'fbank'
si_config['data_folder'] = "/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/"
# si_config['data_folder'] = "/home/muncok/DL/projects/sv_experiments/vad/reddots_vad_cut_1sec/"

In [5]:
import torch
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

def lda_on_tensor(tensor, lda):
    return torch.from_numpy(lda.transform(tensor.numpy()).astype(np.float32))

def embeds_utterance(opt, val_dataloader, model, lda=None):
    val_iter = iter(val_dataloader)
    model.eval()
    splice_dim = opt['splice_frames']
    embeddings = []
    labels = []
    for batch in tqdm_notebook(val_iter, total=len(val_iter)):
        x, y = batch
        time_dim = x.size(2)
        split_points = range(0, time_dim-(splice_dim+12), splice_dim)
        model_outputs = []
        for point in split_points:
            x_in = Variable(x.narrow(2, point, splice_dim+12))
            if not opt['no_cuda']:
                x_in = x_in.cuda()
            model_outputs.append(model.embed(x_in).cpu().data)
        model_output = torch.stack(model_outputs, dim=0)
        model_output = model_output.mean(0)
        if lda is not None:
            model_output = torch.from_numpy(lda.transform(model_output.numpy()).astype(np.float32))
        embeddings.append(model_output)
        labels.append(y.numpy())
    embeddings = torch.cat(embeddings)
    labels = np.hstack(labels)
    return embeddings, labels

### Load Dataset

In [7]:
ndx_file = pd.DataFrame(ndx.file.unique().tolist(), columns=['file'])
ndx_dataset = SpeechDataset.read_df(si_config, ndx_file, "test")
val_dataloader = init_default_loader(si_config, ndx_dataset, shuffle=False) 
ndx_embeddings, _ = embeds_utterance(si_config, val_dataloader, model, lda)

### Load Model

In [10]:
from sv_system.model.TDNN import TdnnModel
model = TdnnModel(si_config, 70, embed_mode=True)
model.load("../models/compare_train_methods/reddots/si_reddots_TdnnModel_3s_0.1s_fbank_full_frame.pt")
model.cuda()
# lda = pickle.load(open("models/lda/si_reddots_0.2s_random_2_lda.pkl", "rb"))
lda = None

models/commands/equal_num_102spk_dot1.pt is loaded


### Embedings

In [12]:
val_dataloader = init_embed_loaders(options, val_df)
embedings, labels = embeds(options, val_dataloader, model) # embeddings: sample x emb_size

HBox(children=(IntProgress(value=0, max=581), HTML(value='')))

### LDA Estimator

In [13]:
n_test = 100 # for test samples

In [14]:
n_samples = embedings.shape[0]
clf = LDA()
random_idx = np.random.permutation(np.arange(0,n_samples))
train_X, train_y = embedings[random_idx[:n_samples-n_test]], labels[random_idx[:n_samples-n_test]]
test_X, test_y = embedings[random_idx[-n_test:]], labels[random_idx[-n_test:]]
clf.fit(train_X, train_y)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [15]:
score = clf.score(test_X, test_y)
print(score) # test_score

0.1


In [16]:
lda_out = "models/lda/{}_splice_lda.pkl".format(options.input.split('/')[-1][:-3])
pickle.dump(clf, open(lda_out, "wb"))